In [86]:
import pygetwindow as gw
from Quartz import CGWindowListCopyWindowInfo, kCGWindowListOptionOnScreenOnly, kCGNullWindowID
from AppKit import NSWorkspace, NSApplication, NSRunningApplication
from Quartz.CoreGraphics import CGRectMake
import objc
import subprocess
from dotenv import load_dotenv
from openai import OpenAI
import os
import json

In [91]:
# Function to find a window by application name
def find_window(app_name):
    windows = CGWindowListCopyWindowInfo(kCGWindowListOptionOnScreenOnly, kCGNullWindowID)
    for window in windows:
        # print(window)
        # if 'kCGWindowName' in window and 'kCGWindowOwnerName' in window:
            # print(window['kCGWindowOwnerName'])
        if window['kCGWindowOwnerName'] == app_name:
            return window
    return None

# Resize the window
def resize_window(window, x_bound, y_bound, width, height):
    # Get the window ID and current bounds
    # window_id = window['kCGWindowNumber']
    # bounds = window['kCGWindowBounds']
    
    # New position and size
    new_position = (x_bound, y_bound)
    new_size = (width, height)
    
    # Use an AppleScript command to resize the window
    script = f"""
    tell application "System Events"
        set the position of window 1 of process "{window["kCGWindowOwnerName"]}" to {{{new_position[0]}, {new_position[1]}}}
        set the size of window 1 of process "Discord" to {{{new_size[0]}, {new_size[1]}}}
    end tell
    """
    try:
        subprocess.run(['osascript', '-e', script])
        return "Success"
    except Exception as e:
        return e

In [15]:
gw.getAllTitles()

['AXVisualSupportAgent ',
 'Stats ',
 'Stats ',
 'Stats ',
 'Stats ',
 'Control Center ',
 'Control Center ',
 'Control Center ',
 'Control Center ',
 'Control Center ',
 'Ollama ',
 'Rectangle ',
 'Window Server Menubar',
 'Dock ',
 'Visual Studio Code ',
 'Discord ',
 'Finder ',
 'Notes ']

In [37]:
# Example usage
app_name = "Discord"  # Replace with your application's name
window = find_window(app_name)
print(window)

{
    kCGWindowAlpha = 1;
    kCGWindowBounds =     {
        Height = 600;
        Width = 940;
        X = 29;
        Y = 50;
    };
    kCGWindowIsOnscreen = 1;
    kCGWindowLayer = 0;
    kCGWindowMemoryUsage = 2176;
    kCGWindowNumber = 64164;
    kCGWindowOwnerName = Discord;
    kCGWindowOwnerPID = 15814;
    kCGWindowSharingState = 0;
    kCGWindowStoreType = 1;
}


In [70]:
if window:
    resize_window(window, 80, 150, 1900, 1100)
else:
    print(f"No window found for application {app_name}")

(80, 150) (1900, 1100)


In [ ]:
# We will need to figure out a way to map from things like Visual Studio Code to the actual name we have to use(Electron) and also deal with multiple electron applications


In [92]:
def full_resize(window_name, x=0, y=0, w=1000, h=1000):
    window = find_window(window_name)
    if window:
        return resize_window(window, x, y, w, h)
    else:
        return (f"No window found for application {window_name}")

In [80]:
full_resize("Discord", 800, 150, 1900, 1100)

'Success'

# Function calling implementation

In [89]:
load_dotenv()
os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [90]:
# defining the tools(functions)
tools = [
    {
        "type": "function",
        "function": {
            "name": "full_resize",
            "description": "Resize a window to the specified dimensions",
            "parameters": {
                "type": "object",
                "properties": {
                    "window_name": {
                        "type": "string",
                        "description": "The name of the window to resize"
                    },
                    "x": {
                        "type": "number",
                        "description": "The x-coordinate of the window"
                    },
                    "y": {
                        "type": "number",
                        "description": "The y-coordinate of the window"
                    },
                    "w": {
                        "type": "number",
                        "description": "The width of the window"
                    },
                    "h": {
                        "type": "number",
                        "description": "The height of the window"
                    }
                },
                "required": ["window_name", "x", "y", "w", "h"]
            }
        }
    }
]
    # {
    #     "type": "function",
    #     "function": {
    #         "name": "find_window",
    #         "description": "Find a window by application name",
    #         "parameters": {
    #             "type": "object",
    #             "properties": {
    #                 "app_name": {
    #                     "type": "string",
    #                     "description": "The name of the application to find"
    #                 }
    #             },
    #             "required": ["app_name"]
    #         }
    #     }
    # }

In [99]:
available_functions = {
    "full_resize": full_resize
}

def pipeline(input):
    messages = [{"role": "user", "content": input}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    # print(response)
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function = available_functions[function_name]
            parameters = json.loads(tool_call.function.arguments)
            
            func_response = function(
                **parameters
            )
            
            
    


In [105]:
# Thoughts on this implementation(7/31/24):
# 
# Works really well--but. It never full brings the windows all the way to the right edge.
# This will be interesting to look into and I expect it will get more refined as we work 
# with different machines and resolutions as well as applications.
# 
# Interestingly it's tolerance changes based on it's starting location, we may need to 
# give these as parameters to another function that can get called before going to the 
# main function.
# 
# This will likely be an interesting ongoing problem we will have to mess with to figure 
# out a method to make it stay consistent even as user queries get increasingly more
# complex(ie: "Move my discord window to the middle bottom of my top monitor")
# 
# Another interesting thing is that in MacOS windows that aren't on your current monitor
# (or if you are fullscreened) don't show up in the list of windows. I have not tested
# if they can still be manipulated, but I would assume they can't.

pipeline(input="put discord at the right of the screen")